In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv('../data/sample_employee_data.csv')
df.head(3)

,id,first_name,last_name,age,is_resident,is_unemployed,has_insufficient_means,vision_impaired,is_widow_or_widower,is_not_cohabiting,...,is_providing_care,husband_in_prison,is_single_woman,is_orphan,department,hiring_manager,number_of_children,date_of_hire,employment_status,annual_income
0,1,Mary,Smith,32,1,0,0,0,0,0,...,0,0,0,0,Marketing,John Doe,2,2019-05-15,Full-time,55000.0
1,2,John,Johnson,45,1,0,0,0,0,0,...,0,0,0,0,Engineering,Jane Smith,3,2020-02-28,Full-time,72000.0
2,3,Jennifer,Brown,28,1,0,0,1,0,1,...,0,0,1,0,HR,David Williams,0,2021-09-10,Part-time,38000.0


In [3]:
with open('../data/sample_schema.json', 'r') as f:
    sample_schema = json.load(f) # this is the schema for sample data

In [4]:
import os, sys
sys.path.append('..')

In [5]:
from schema_agent import schema_agent

In [6]:
rules = """
| Scheme | Scheme Rules |
|--------|--------------|
| **Unemployment Assistance** | 1. Resident of the State. 2. Unemployed but capable of and available for work. 3. Have insufficient means. |
| **Old Age Assistance** | 1. Resident of the State. 2. Aged 66 years or over. 3. Have insufficient means. |
| **Blind Pension** | 1. Resident of the State. 2. Aged 18 years or over. 3. Have a vision impairment. 4. Have insufficient means. |
| **Widow’s and Widower’s (Non-Contributory) Pension** | 1. Resident of the State. 2. Widow or widower not cohabiting. 3. Have insufficient means. |
| **Deserted Wife’s Allowance** | 1. Resident of the State. 2. Deserted by husband. 3. Have insufficient means. 4. Not cohabiting. |
| **Carer's Allowance** | 1. Resident of the State. 2. Providing full-time care and attention to a person in need. 3. Have insufficient means. |
| **Prisoner's Wife Allowance** | 1. Resident of the State. 2. Husband in prison. 3. Have insufficient means. 4. Not cohabiting. |
| **Single Woman’s Allowance** | 1. Resident of the State. 2. Aged 58 years or over. 3. Single woman living alone. 4. Have insufficient means. |
| **Orphan’s (Non-Contributory) Pension** | 1. Resident of the State. 2. Under 18 years of age. 3. Orphaned. 4. Have insufficient means. |
"""

In [7]:
required_schema = schema_agent(sample_schema, rules)

In [8]:
required_cols = [col['name'] for col in required_schema['columns']]
required_cols

['age',
 'is_resident',
 'is_unemployed',
 'has_insufficient_means',
 'vision_impaired',
 'is_widow_or_widower',
 'is_not_cohabiting',
 'is_deserted_by_husband',
 'is_providing_care',
 'husband_in_prison',
 'is_single_woman',
 'is_orphan']

In [9]:
required_df = df[required_cols]
required_df.head(3)

,age,is_resident,is_unemployed,has_insufficient_means,vision_impaired,is_widow_or_widower,is_not_cohabiting,is_deserted_by_husband,is_providing_care,husband_in_prison,is_single_woman,is_orphan
0,32,1,0,0,0,0,0,0,0,0,0,0
1,45,1,0,0,0,0,0,0,0,0,0,0
2,28,1,0,0,1,0,1,0,0,0,1,0


In [10]:
import ollama

In [11]:
client = ollama.Client('http://localhost:11434')

In [12]:
def create_user_message(row: pd.Series, schema: dict) -> str:
    """
    Agent to write an introduction for the user based on the data provided and the schema.
    """
    system_message = f"Your job is to introduce yourself as the user based on the data provided and the schema. Make sure to include all the information (and no more) and be consise and clear. This is the schema: \n\n{schema}"
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': str(row)},
    ]
    response = client.chat('llama3', messages)
    return response['message']['content']

user_message = create_user_message(required_df.iloc[0], required_schema)
print(user_message)

Hello! My name is not provided in the given schema. However, I can introduce myself based on my characteristics.

I am a 32-year-old individual who resides in the state. I am employed and do not have insufficient means to support myself. I do not have any vision impairment and am not a widow or widower. I am currently cohabiting with someone and have not been deserted by my partner. I am not providing full-time care to anyone, nor is my husband in prison. Additionally, I am not a single woman and am not an orphan.

That's me in a nutshell!


In [13]:
def assess_scheme(user_message: str, rules: str) -> str:
    """
    Agent to assess the user message based on the rules provided.
    """
    system_message = f"""
    You are a helpful AI assistant who is an expert at insurance schemes. 
    Your job is to determine which scheme a user falls into based on their personal information. 
    You must use the following rules to determine the scheme: \n\n{rules}
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message},
    ]
    response = client.chat('llama3', messages)
    return response['message']['content']

scheme_analysis = assess_scheme(user_message, rules)
print(scheme_analysis)

Thank you for introducing yourself!

Based on the characteristics you've shared, it seems that none of the schemes listed match your profile. You're employed, have sufficient means, and don't fit into any of the categories mentioned (e.g., unemployment, old age, blindness, widowhood, desertion, caregiving, or imprisonment).

It appears that you fall outside the scope of the provided insurance schemes. If you're eligible for other benefits or programs, it would depend on specific circumstances not mentioned here.

Please let me know if there's anything else I can help with!


In [14]:
def scheme_result(scheme_analysis: str) -> str:
    """
    Agent to extract the result of the scheme analysis.
    """
    system_message = f"""
    You are a helpful AI assistant who is an expert at insurance schemes. 
    Your job is to output the scheme that the user is matched to based on the provided analysis.
    Only reply with the scheme name and nothing else. Mkae sure to strip away any markdown formatting such as '**'. If the user is not compatible with any scheme, reply with 'No Scheme'.
    You must use the scheme names from the following rules: \n\n{rules}
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': scheme_analysis},
    ]
    response = client.chat('llama3', messages)
    return response['message']['content']

scheme = scheme_result(scheme_analysis)
print(scheme)

No Scheme


In [15]:
def get_scheme(row: pd.Series, schema: dict, rules: str) -> str:
    """
    Agent to get the scheme based on the row data.
    """
    user_message = create_user_message(row, schema)
    scheme_analysis = assess_scheme(user_message, rules)
    scheme = scheme_result(scheme_analysis)
    return scheme

scheme = get_scheme(required_df.iloc[2], required_schema, rules)
print(scheme)

No Scheme


In [16]:
df.head(5).apply(lambda row: get_scheme(row, required_schema, rules), axis=1)

0    No Scheme
1    No Scheme
2    No Scheme
3    No Scheme
4    No Scheme
dtype: object